In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm_notebook

df_larval = pd.read_csv(os.path.join('..', 'data', 'breeding-sites', 'larval-survey-en.csv'))
df_larval.head()

,date,province,district,subdist,village,hi,ci,bi,province_en,district_en,subdist_en
0,2017-6,กระบี่,คลองท่อม,ห้วยน้ำขาว,ห้วยน้ำขาว,22.5,6.47,22.5,Krabi,Khlong Thom,Huai Nam Khao
1,2017-6,กระบี่,คลองท่อม,ห้วยน้ำขาว,น้ำร้อน,15.0,4.52,17.5,Krabi,Khlong Thom,Huai Nam Khao
2,2017-6,นครศรีธรรมราช,ฉวาง,ห้วยปริก,ปากกา,17.0,3.18,17.0,Nakhon Si Thammarat,Chawang,Huai Prik
3,2017-6,นครศรีธรรมราช,ทุ่งสง,ควนกรด,หนองเสม็ด,25.0,4.51,27.5,Nakhon Si Thammarat,Thung Song,Khuan Krot
4,2017-6,นครศรีธรรมราช,ทุ่งสง,นาโพธิ์,เกาะปราง,15.0,8.95,42.5,Nakhon Si Thammarat,Thung Song,Na Pho


## Shapefile

In [2]:
with open(os.path.join('..', 'data','shapefiles','Nakhon-Si-Thammarat.geojson')) as f:
    data = json.load(f)

In [3]:
for i, feature in enumerate(data['features']):
    prop = feature['properties']

    province = prop['PV_TN']
    district = prop['AP_TN']
    subdist = prop['TB_TN']
    
    df_tmp = df_larval.loc[(df_larval['province'] == province) & 
                       (df_larval['district'] == district)]
    province_en, district_en = df_tmp[['province_en','district_en']].values[0]
    
    prop['PV_EN'] = province_en
    prop['AP_EN'] = district_en

In [4]:
data['features'][2]['properties']

{'AP_EN': 'Mueang Nakhon Si Thammarat',
 'AP_IDN': '8001',
 'AP_TN': 'เมืองนครศรีธรรมราช',
 'PV_EN': 'Nakhon Si Thammarat',
 'PV_IDN': '80',
 'PV_TN': 'นครศรีธรรมราช',
 'TB_EN': 'Pak Phun',
 'TB_IDN': '800120',
 'TB_TN': 'ปากพูน'}

In [5]:
with open(os.path.join('..', 'data', 'shapefiles', 'Nakhon-Si-Thammarat-en.geojson'), 'w') as FILE:
    json.dump(data, FILE, indent=4, ensure_ascii=False, sort_keys=True)

## Dictonary file

In [6]:
province_entry = []
for feature in data['features']:
    prop = feature['properties']
    
    province_entry.append([
        prop['PV_TN'],
        prop['AP_TN'],
        prop['TB_TN'],
        prop['PV_EN'],
        prop['AP_EN'],
        prop['TB_EN'],
    ])
    
province_entry = pd.DataFrame.from_records(province_entry, columns=['province_th', 'district_th', 'subdist_th',
                                                                   'province_en', 'district_en', 'subdist_en'])

province_entry.to_csv(os.path.join('..', 'data', 'shapefiles', 'Nakhon-Si-Thammarat-dictionary.csv'))
province_entry.head()

,province_th,district_th,subdist_th,province_en,district_en,subdist_en
0,นครศรีธรรมราช,พรหมคีรี,ทอนหงส์,Nakhon Si Thammarat,Phrom Khiri,Thon Hong
1,นครศรีธรรมราช,ท่าศาลา,โมคลาน,Nakhon Si Thammarat,Tha Sala,Mo Khlan
2,นครศรีธรรมราช,เมืองนครศรีธรรมราช,ปากพูน,Nakhon Si Thammarat,Mueang Nakhon Si Thammarat,Pak Phun
3,นครศรีธรรมราช,พิปูน,พิปูน,Nakhon Si Thammarat,Phipun,Phipun
4,นครศรีธรรมราช,ท่าศาลา,ดอนตะโก,Nakhon Si Thammarat,Tha Sala,Don Tako


## Detection file

In [7]:
with open(os.path.join('..', 'data','breeding-sites','detection.geojson')) as f:
    detection = json.load(f)

In [8]:
for feature in tqdm_notebook(detection['features']):
    prop = feature['properties']
    
    province = prop['province']
    district = prop['district']
    subdist = prop['subdist']

    df_tmp = province_entry.loc[
        (province_entry['province_th'] == province) & 
        (province_entry['district_th'] == district) &
        (province_entry['subdist_th'] == subdist)
    ]
    
    province_en, district_en, subdist_en = df_tmp[['province_en','district_en', 'subdist_en']].values[0]
    
    prop['province_en'] = province_en
    prop['district_en'] = district_en
    prop['subdist_en'] = subdist_en

HBox(children=(IntProgress(value=0, max=42999), HTML(value='')))

In [9]:
with open(os.path.join('..', 'data','breeding-sites','detection-en.geojson'), 'w') as FILE:
    json.dump(detection, FILE, indent=4, ensure_ascii=False, sort_keys=True)